In [1]:
# Copyright (c) Cyril Zakka.
# All rights reserved.

# This source code is licensed under the license found in the
# LICENSE file in the root directory of this source tree.
# --------------------------------------------------------
# Modified from:
# DeiT: https://github.com/facebookresearch/deit
# BEiT: https://github.com/microsoft/unilm/tree/master/beit
# MAE: https://github.com/facebookresearch/MAE
# --------------------------------------------------------
import argparse
import datetime
import json
from torch.utils.tensorboard import SummaryWriter
from PIL import Image
import numpy as np
import os
import time
from pathlib import Path

import torch
import torch.backends.cudnn as cudnn
import torchvision.transforms as transforms

import timm.optim.optim_factory as optim_factory

import util.misc as misc
from util.misc import NativeScalerWithGradNormCount as NativeScaler


import model_mae3d
from engine_pretrain3d import train_one_epoch




def get_args_parser():
    parser = argparse.ArgumentParser('MAE3D pre-training', add_help=False)
    parser.add_argument('--batch_size', default=8, type=int,
                        help='Batch size per GPU (effective batch size is batch_size * accum_iter * # gpus')
    parser.add_argument('--epochs', default=800, type=int)
    parser.add_argument('--accum_iter', default=1, type=int,
                        help='Accumulate gradient iterations (for increasing the effective batch size under memory constraints)')

    # Model parameters
    parser.add_argument('--model', default='mae3d_vit_base_stride8_patch8', type=str, metavar='MODEL',
                        help='Name of model to train')
    parser.add_argument('--input_size', default=128, type=int,
                        help='images input size')
    parser.add_argument('--mask_ratio', default=0.5, type=float,
                        help='Masking ratio (percentage of removed patches).')
    parser.add_argument('--norm_pix_loss', action='store_true',
                        help='Use (per-patch) normalized pixels as targets for computing loss')
    parser.set_defaults(norm_pix_loss=True)

    # Optimizer parameters
    parser.add_argument('--weight_decay', type=float, default=0.05,
                        help='weight decay (default: 0.05)')
    parser.add_argument('--lr', type=float, default=1e-5, metavar='LR',
                        help='learning rate (absolute lr)')
    parser.add_argument('--blr', type=float, default=1e-5, metavar='LR',
                        help='base learning rate: absolute_lr = base_lr * total_batch_size / 256')
    parser.add_argument('--min_lr', type=float, default=0., metavar='LR',
                        help='lower lr bound for cyclic schedulers that hit 0')
    parser.add_argument('--warmup_epochs', type=int, default=50, metavar='N',
                        help='epochs to warmup LR')

    # Dataset parameters
    parser.add_argument('--data_path', default='/home/cyril/Datasets/MAE/', type=str,
                        help='dataset path')
    parser.add_argument('--num_segments', default=8, type=int)
    parser.add_argument('--frames_per_segment', default=16, type=int)
    parser.add_argument('--output_dir', default='/data2/zhanghao/mae_project/pretrain_model/oasis4_35_pre800_freeze_patch8/',
                        help='path where to save, empty for no saving')
    parser.add_argument('--log_dir', default='./mae-main/pretrain_model/oasis4_35_pre800_freeze_patch8',
                        help='path where to tensorboard log')
    parser.add_argument('--device', default='cuda:1',
                        help='device to use for training / testing')
    parser.add_argument('--seed', default=0, type=int)
    parser.add_argument('--resume', default='',
                        help='resume from checkpoint')
    parser.add_argument('--start_epoch', default=0, type=int, metavar='N',
                        help='start epoch')
    parser.add_argument('--num_workers', default=10, type=int)
    parser.add_argument('--pin_mem', action='store_true',
                        help='Pin CPU memory in DataLoader for more efficient (sometimes) transfer to GPU.')
    parser.add_argument('--no_pin_mem', action='store_false', dest='pin_mem')
    parser.set_defaults(pin_mem=True)

    # distributed training parameters
    parser.add_argument('--world_size', default=1, type=int,
                        help='number of distributed processes')
    parser.add_argument('--local_rank', default=-1, type=int)
    parser.add_argument('--dist_on_itp', action='store_true')
    parser.add_argument('--dist_url', default='env://',
                        help='url used to set up distributed training')

    return parser


In [2]:
from util.oasis4_35dataset import *
train_paths = getPathList()
train_loader = getDataloader_all(
    train_paths, B1=2)
len(train_loader)
#(1,1,160,224,224)

207

In [3]:
def main(args):
    # misc.init_distributed_mode(args)

    # print('job dir: {}'.format(os.path.dirname(os.path.realpath(__file__))))
    print("{}".format(args).replace(', ', ',\n'))

    device = torch.device(args.device)

    # fix the seed for reproducibility
    seed = args.seed + misc.get_rank()
    torch.manual_seed(seed)
    np.random.seed(seed)

    cudnn.benchmark = True

    if args.log_dir is not None:
        os.makedirs(args.log_dir, exist_ok=True)
        log_writer = SummaryWriter(log_dir=args.log_dir)

    # define the model
    model = model_mae3d.__dict__[args.model](
        num_frames=int(args.num_segments*args.frames_per_segment), norm_pix_loss=args.norm_pix_loss)

    #freeze patch_embded param
    for name,param in model.named_parameters():
        if name=='patch_embed.proj.weight':
                param.requires_grad = False
                print('freeze ',name)
        if name=='patch_embed.proj.bias':
                param.requires_grad = False
                print('freeze ',name)

    model.to(device)

    model_without_ddp = model
    print("Model = %s" % str(model_without_ddp))

    eff_batch_size = args.batch_size * args.accum_iter * misc.get_world_size()
    
    # if args.lr is None:  # only base_lr is specified
    #     args.lr = args.blr * eff_batch_size / 256

    # print("base lr: %.2e" % (args.lr * 256 / eff_batch_size))
    print("actual lr: %.2e" % args.lr)

    print("accumulate grad iterations: %d" % args.accum_iter)
    print("effective batch size: %d" % eff_batch_size)

    # if args.distributed:
    #     model = torch.nn.parallel.DistributedDataParallel(model, device_ids=[args.gpu], find_unused_parameters=True)
    #     model_without_ddp = model.module
    
    # following timm: set wd as 0 for bias and norm layers
    param_groups = optim_factory.param_groups_weight_decay(
        model_without_ddp, args.weight_decay)

    optimizer = torch.optim.AdamW(param_groups, lr=args.lr, betas=(0.9, 0.95))
    print(optimizer)
    loss_scaler = NativeScaler()

    misc.load_model(args=args, model_without_ddp=model_without_ddp, optimizer=optimizer, loss_scaler=loss_scaler)

    print(f"Start training for {args.epochs} epochs")
    start_time = time.time()
    for epoch in range(args.start_epoch, args.epochs):
        # if args.distributed:
        #     train_loader.sampler.set_epoch(epoch)
        train_stats = train_one_epoch(
            model, train_loader,
            optimizer, device, epoch, loss_scaler,
            log_writer=log_writer,
            args=args
        )
        if args.output_dir and (epoch % 10 == 0 or epoch + 1 == args.epochs):
            misc.save_model(
                args=args, model=model, model_without_ddp=model_without_ddp, optimizer=optimizer,
                loss_scaler=loss_scaler, epoch=epoch)

        log_stats = {**{f'train_{k}': v for k, v in train_stats.items()},
                        'epoch': epoch,}

        if args.output_dir and misc.is_main_process():
            if log_writer is not None:
                log_writer.flush()
            with open(os.path.join(args.output_dir, "log.txt"), mode="a", encoding="utf-8") as f:
                f.write(json.dumps(log_stats) + "\n")    

    total_time = time.time() - start_time
    total_time_str = str(datetime.timedelta(seconds=int(total_time)))
    print('Training time {}'.format(total_time_str))


In [4]:
args = get_args_parser()
args = args.parse_args(args=[])
if args.output_dir:
    Path(args.output_dir).mkdir(parents=True, exist_ok=True)
main(args)

Namespace(batch_size=8,
epochs=800,
accum_iter=1,
model='mae3d_vit_base_stride8_patch8',
input_size=128,
mask_ratio=0.5,
norm_pix_loss=True,
weight_decay=0.05,
lr=1e-05,
blr=1e-05,
min_lr=0.0,
warmup_epochs=50,
data_path='/home/cyril/Datasets/MAE/',
num_segments=8,
frames_per_segment=16,
output_dir='/data2/zhanghao/mae_project/pretrain_model/oasis4_35_pre800_freeze_patch8/',
log_dir='./mae-main/pretrain_model/oasis4_35_pre800_freeze_patch8',
device='cuda:1',
seed=0,
resume='',
start_epoch=0,
num_workers=10,
pin_mem=True,
world_size=1,
local_rank=-1,
dist_on_itp=False,
dist_url='env://')
初始化参权重
freeze  patch_embed.proj.weight
freeze  patch_embed.proj.bias
Model = MAE3D(
  (patch_embed): PatchEmbed3D(
    (proj): Conv3d(1, 768, kernel_size=(8, 8, 8), stride=(8, 8, 8))
    (norm): Identity()
  )
  (blocks): ModuleList(
    (0): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias

Exception in thread Thread-3:
Traceback (most recent call last):
  File "/home/zhanghao/.conda/envs/ZH/lib/python3.9/threading.py", line 973, in _bootstrap_inner
    self.run()
  File "/home/zhanghao/.conda/envs/ZH/lib/python3.9/site-packages/tensorboard/summary/writer/event_file_writer.py", line 234, in run
    self._record_writer.write(data)
  File "/home/zhanghao/.conda/envs/ZH/lib/python3.9/site-packages/tensorboard/summary/writer/record_writer.py", line 40, in write
    self._writer.write(header + header_crc + data + footer_crc)
  File "/home/zhanghao/.conda/envs/ZH/lib/python3.9/site-packages/tensorboard/compat/tensorflow_stub/io/gfile.py", line 527, in write
    self.fs.append(self.filename, file_content, self.binary_mode)
  File "/home/zhanghao/.conda/envs/ZH/lib/python3.9/site-packages/tensorboard/compat/tensorflow_stub/io/gfile.py", line 150, in append
    self._write(filename, file_content, "ab" if binary_mode else "a")
  File "/home/zhanghao/.conda/envs/ZH/lib/python3.9/sit

# 继续训练